# Notebook 12.e: A Glimpse of Calculus

> The limit of a function is a fundamental concept in calculus and analysis concerning the behavior of that function near a particular input.
>
> — Wikipedia


## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
- Interpret the first difference as the average rate of change (or average sensitivity).
- Visualize the average rate of change as the slope of a secant line.
- Build an intuition for the limit by visualizing how the secant line approaches the tangent line as the interval shrinks.

## 📚 Prerequisites

This notebook builds on concepts from all the previous lessons in the 12.x series.

*Estimated Time: 40 minutes*

---

[Return to Table of Contents](https://colab.research.google.com/github/sguy/programming-and-problem-solving/blob/main/notebooks/table-of-contents.ipynb)

## Introduction: How to Measure Speed?

Imagine you're a scout at a baseball game, and you need to measure the speed of a pitcher's fastball. How would you do it?

**Attempt 1: The Low-Tech Way**
You could start a timer when the pitcher releases the ball and stop it when the catcher catches it. The distance from the pitcher's mound to home plate is 60.5 feet. If the ball takes 0.4 seconds to travel that distance, you can calculate the speed:
`Speed = Distance / Time = 60.5 feet / 0.4 s = 151.25 feet/sec` (about 103 mph).

But is that the *true* speed of the pitch? Not really. The ball is fastest when it leaves the pitcher's hand and slows down due to air resistance. What you've calculated is the **average speed** over the entire path.

If we were to plot the ball's position over time, this measurement is equivalent to finding the slope of a line connecting the start and end points of the journey. This line, which connects two points on a curve, is called a **secant line**. The slope of this secant line gives you the *average rate of change* over that interval.

You can also think of this as **average sensitivity**. It answers the question: on average, how sensitive is the ball's position to a change in time? A high value means the position changes a lot for a little bit of time, while a low value means it doesn't change much.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define a function that models the distance a pitched baseball travels.
# It's a quadratic function to simulate the ball slowing down due to air resistance.
# We've chosen the numbers so that at t=0.4s, the distance is 60.5 feet.
def pitch_distance(t):
    initial_velocity = 155  # ft/s (a bit faster than the average)
    drag_factor = 18.75
    return initial_velocity * t - 0.5 * drag_factor * t**2

# Define the start and end points for our measurement, as in the example.
t1, t2 = 0.0, 0.4
pos1, pos2 = pitch_distance(t1), pitch_distance(t2)

# --- Plotting ---
plt.figure(figsize=(10, 6))
# Create a range of time values for our plot, centered around our measurement interval
t_vals = np.linspace(0, 0.5, 200)
pos_vals = pitch_distance(t_vals)

# Plot the curve representing the ball's distance over time
plt.plot(t_vals, pos_vals, label="Ball's Distance vs. Time")

# Plot the secant line connecting the start and end points of the measurement
plt.plot([t1, t2], [pos1, pos2], 'r--o', label='Secant Line')

# Add labels and annotations
plt.title('The Secant Line Represents Average Speed of a Pitch', fontsize=16)
plt.xlabel('Time (seconds)')
plt.ylabel('Distance Traveled (feet)')
plt.legend()
plt.grid(True)
plt.annotate('Release Point (P1)', (t1, pos1), textcoords="offset points", xytext=(20, -20), ha='center', fontsize=12, color='red')
plt.annotate('Home Plate (P2)', (t2, pos2), textcoords="offset points", xytext=(-30, 20), ha='center', fontsize=12, color='red')

plt.show()

# Let's calculate the average speed from the secant line slope
average_speed = (pos2 - pos1) / (t2 - t1)
print(f"The slope of the secant line (average speed) is: {average_speed:.2f} feet/sec")

**Attempt 2: A Better Way**
To get a better estimate of the *peak* speed, you could set up two laser gates just one foot apart, right in front of the pitcher. You measure the tiny amount of time it takes for the ball to pass between them. This gives you the average speed over a much smaller interval, which is a much better approximation of the ball's speed as it leaves the pitcher's hand.

On our graph, this would be like moving our two points much closer together. The secant line connecting them would now be a much better match for the curve's steepness at that location.

**The Calculus Way**
A radar gun does something even more precise. It uses the Doppler effect to measure the speed over an incredibly tiny interval of time, giving a very, very good approximation of the **instantaneous speed**.

This is the core idea of differential calculus. To get from an *average* rate of change (the slope of a secant line) to an *instantaneous* rate of change, we need to make the interval between our two points smaller and smaller. In the language of our graph, we are sliding one point along the curve until it is infinitesimally close to the other.

As the interval shrinks to zero, the secant line becomes a **tangent line**—a line that touches the curve at just one single point. The slope of that tangent line is the instantaneous rate of change, also known as the **derivative**.

In this notebook, we'll explore this process visually and computationally, using our `height_model` to see how the secant line approaches the tangent line as we shrink our interval, `h`.

## 🛠️ Our Detective Kit: The Tools

Just like in our previous investigations, we need our standard set of tools. Below are the helper functions we've developed in the 12.x series. We'll need them for the challenges ahead.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def get_function_values(func, domain):
    codomain = []
    for x in domain:
        codomain.append(func(x))
    return codomain

def calculate_differences(sequence):
    if not sequence or len(sequence) < 2:
        return [np.nan] * len(sequence)
    differences = [np.nan]
    for i in range(1, len(sequence)):
        if np.isnan(sequence[i]) or np.isnan(sequence[i-1]):
            differences.append(np.nan)
        else:
            differences.append(sequence[i] - sequence[i-1])
    return differences

def velocity_estimate(func, t, h):
    return (func(t + h) - func(t)) / h

def extrapolation_error(func, t, h, t2):
    v = velocity_estimate(func, t, h)
    h_predicted = func(t) + (t2 - t) * v
    h_actual = func(t2)
    return abs(h_actual - h_predicted)

### 🎯 Mini-Challenge: Calculating the Slope of the Secant

Before we visualize the secant line, your first task is to write a Python function that calculates the slope of the secant line for any given function.

**Your Goal:** Complete the `calculate_secant_slope(func, x, h)` function. It should:
1. Take a function `func`, a point `x`, and a small interval `h` as input.
2. Calculate the coordinates of two points on the function's curve:
   - Point 1: `(x, func(x))`
   - Point 2: `(x + h, func(x + h))`
3. Calculate and return the slope between these two points. Remember the slope formula: `(y2 - y1) / (x2 - x1)`.

<details>
<summary>Hint: Implementing the function</summary>

You'll need to call the function `func` twice inside your `calculate_secant_slope` function to get the two y-values.
</details>
<details>
<summary>Hint: Calculating the slope</summary>

Remember the slope formula: `slope = (y2 - y1) / (x2 - x1)`. In our case, `x2 - x1` is simply `(x + h) - x`, which simplifies to `h`.
</details>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def height_model(time):
    """Calculates the height of our ball at a given time."""
    initial_height = 10
    initial_velocity = 50
    gravity = 10
    return initial_height + (initial_velocity * time) - (0.5 * gravity * time**2)

def calculate_secant_slope(func, x, h):
    # YOUR CODE HERE
    pass

# We've provided a more advanced plotting function for you
def plot_secant_line(t, h):
    # 1. Define the points
    t1 = t
    h1 = height_model(t1)
    t2 = t + h
    h2 = height_model(t2)

    # 2. Calculate the slope (average velocity)
    avg_velocity = (h2 - h1) / (t2 - t1)

    # 3. Set up the plot
    plt.figure(figsize=(10, 6))
    
    # 4. Plot the original function
    time_vals = np.linspace(0, 10, 200)
    height_vals = height_model(time_vals)
    plt.plot(time_vals, height_vals, label='Ball Trajectory')

    # 5. Plot the two points
    plt.plot([t1, t2], [h1, h2], 'ro')

    # 6. Plot the extended secant line
    # Define a wider range for the line
    t_line = np.linspace(t - 2, t + 2, 10)
    # Use point-slope form: y - y1 = m(x - x1) => y = m(x - x1) + y1
    h_line = avg_velocity * (t_line - t1) + h1
    plt.plot(t_line, h_line, 'r--', label=f'Secant line for h={h}')
    
    # 7. Add labels and title
    plt.title(f'Average Velocity from t={t} to t={t+h} is {avg_velocity:.2f} m/s')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Height (meters)')
    plt.legend()
    plt.grid(True)
    # Zoom in on the area of interest
    plt.xlim(t - 1, t + h + 1)
    plt.ylim(h1 - 10, h1 + 30)
    plt.show()


# Use a loop to visualize the secant line as h gets smaller
# Let's find the instantaneous velocity at t=2 seconds
for h_value in [3, 2, 1, 0.5, 0.1, 0.01]:
    slope = calculate_secant_slope(height_model, 2, h_value)
    print(f"For h = {h_value}, the secant slope is: {slope}")
    plot_secant_line(t=2, h=h_value)

<details>
<summary>Click to see a possible solution</summary>

```python
def calculate_secant_slope(func, x, h):
    # Calculate the two y-values
    y1 = func(x)
    y2 = func(x + h)
    # Calculate the slope and return it
    return (y2 - y1) / h

# Use a loop to visualize the secant line as h gets smaller
# Let's find the instantaneous velocity at t=2 seconds
for h_value in [3, 2, 1, 0.5, 0.1, 0.01]:
    slope = calculate_secant_slope(height_model, 2, h_value)
    print(f"For h = {h_value}, the secant slope is: {slope}")
    # plot_secant_line(t=2, h=h_value) # We can comment this out if we just want the numbers
```
</details>

### Analyzing the Limit Visualization

Look at the plots your code generated. As the interval `h` gets smaller, the red secant line pivots, getting closer and closer to the curve at `x=2`. The calculated slope is also converging on a single number.

This limiting line that the secant lines are approaching is called the **tangent line**. It is the line that perfectly touches the curve at that single point. Its slope represents the true **instantaneous rate of change** (also called the **derivative**), which is the core idea of differential calculus.

### ✅ Check Your Understanding

1. The slope of a **secant line** represents the ______ rate of change, while the slope of a **tangent line** represents the ______ rate of change.
<details><summary>Click for the answer</summary>

Answer: *average*, *instantaneous*

</details>

2. As the interval `h` gets smaller and smaller, the slope of the secant line gets closer and closer to the slope of the...?

    * a) x-axis
    * b) tangent line
    * c) y-axis
    * d) secant line itself

<details><summary>Click for the answer</summary>

Answer: **b) tangent line**

</details>

3. If you use the secant method on a function and find that the slopes are converging on the number `-15.0`, what does that number represent?

    * a) The minimum height of the object.
    * b) The total time the object was in the air.
    * c) The instantaneous rate of change at that point.
    * d) The average height of the object.

<details><summary>Click for the answer</summary>

Answer: **c) The instantaneous rate of change at that point.**

</details>

### 🚀 Pro-Tip: Extrapolation and Error

One of the most powerful uses of the instantaneous slope (the derivative) is to make **linear extrapolations**. If you are at a specific time `t` and you know the ball's height `h(t)` and its instantaneous velocity `v`, you can predict its height at a nearby time `t2`:

$$ h_{predicted}(t_2) = h(t) + (t_2 - t) \cdot v$$

This is like saying, "The new height is the old height, plus the velocity times the time elapsed." But how good is this prediction? The error is the difference between the real height and our prediction: `Error = abs(h(t2) - h_predicted(t2))`.

Let's explore this with our `height_model`.

In [ ]:
# Let's visualize the extrapolation and the error

# --- Setup ---
t_tangent = 2
t_extrapolate = 3
h_small = 0.00001 # A very small h to get a good velocity estimate

# --- Calculations ---
# Instantaneous velocity (slope of the tangent) at t_tangent
v_instantaneous = (height_model(t_tangent + h_small) - height_model(t_tangent)) / h_small

# Find the point of tangency
h_tangent = height_model(t_tangent)

# The tangent line is a linear function: y = mx + b, or h = v*t + intercept
# We can find the intercept (b) using the point of tangency: h_tangent = v_instantaneous * t_tangent + intercept
intercept = h_tangent - v_instantaneous * t_tangent
def tangent_line(t):
  return v_instantaneous * t + intercept

# Calculate the actual and predicted heights at the extrapolation point
h_actual_extrap = height_model(t_extrapolate)
h_predicted_extrap = tangent_line(t_extrapolate)
error = abs(h_actual_extrap - h_predicted_extrap)

# --- Plotting ---
plt.figure(figsize=(12, 8))

# Plot the main height_model function
t_vals = np.linspace(1.5, 3.5, 200)
h_vals = height_model(t_vals)
plt.plot(t_vals, h_vals, label='Actual Ball Trajectory', linewidth=2)

# Plot the tangent line
t_tangent_line = np.array([1.5, 3.5])
h_tangent_line = tangent_line(t_tangent_line)
plt.plot(t_tangent_line, h_tangent_line, 'r--', label=f'Tangent Line (Velocity = {v_instantaneous:.2f} m/s)', linewidth=2)

# Mark the point of tangency
plt.plot(t_tangent, h_tangent, 'ro', markersize=10, label='Point of Tangency (t=2s)')

# Mark the actual and predicted points at t_extrapolate
plt.plot(t_extrapolate, h_actual_extrap, 'go', markersize=10, label=f'Actual Height at t=3s ({h_actual_extrap:.2f}m)')
plt.plot(t_extrapolate, h_predicted_extrap, 'yo', markersize=10, label=f'Predicted Height at t=3s ({h_predicted_extrap:.2f}m)')

# Draw the error line
plt.vlines(x=t_extrapolate, ymin=h_predicted_extrap, ymax=h_actual_extrap, color='purple', linestyle=':', linewidth=3, label=f'Extrapolation Error ({error:.2f}m)')

# --- Labels and Annotations ---
plt.title('Visualizing Extrapolation Error', fontsize=16)
plt.xlabel('Time (seconds)')
plt.ylabel('Height (meters)')
plt.legend()
plt.grid(True)
plt.xlim(1.5, 3.5)
plt.ylim(80, 125) # Zoom in on the relevant y-axis section

# Add text labels for clarity
plt.text(t_extrapolate + 0.05, h_actual_extrap, 'Actual Value', verticalalignment='center')
plt.text(t_extrapolate + 0.05, h_predicted_extrap, 'Predicted Value', verticalalignment='center')
plt.text(t_extrapolate + 0.05, (h_actual_extrap + h_predicted_extrap) / 2, 'Error', verticalalignment='center', color='purple')


plt.show()

### 🎯 Mini-Challenge: The Quality of the Extrapolation

Now let's flip the experiment. We'll use a *good* velocity estimate (by fixing `h` to a small value) and see how the extrapolation error changes as we try to predict the ball's height at times further and further away from `t=2`.

**Your Goal:** Plot the extrapolation error as a function of the target time `t2`.

**Steps:**
1. Use the same functions as before (`velocity_estimate`, `extrapolation_error`).
2. Fix your `h` to a small number, like `h = 0.00001`.
3. Create a list of `t2_values` to test. `np.linspace(0, 10, 50)` will give a good range of times.
4. Loop through `t2_values`, call `extrapolation_error` for each `t2` (with `t=2` and your fixed `h`), and store the errors.
5. Create a normal plot of `error` vs. `t2`.

In [ ]:
# YOUR CODE HERE

<details>
<summary>Click to see a possible solution</summary>

```python
h_fixed = 0.00001
t2_values = np.linspace(0, 10, 50)
error_values_2 = []

for t2 in t2_values:
    err = extrapolation_error(height_model, 2, h_fixed, t2)
    error_values_2.append(err)

plt.figure(figsize=(8,6))
plt.plot(t2_values, error_values_2, 'r-o')
plt.title('Extrapolation Error vs. Distance from t=2')
plt.xlabel('t2 (target time in seconds)')
plt.ylabel('Error of Linear Extrapolation from t=2 (meters)')
plt.grid(True)
plt.show()
```

</details>

### Analyzing the Second Plot

This result is just as important! The plot shows that the error is zero at `t=2` (as you'd expect) and grows bigger the further we move away from `t=2`. The shape of the error is parabolic, which again tells us something deep about the nature of the approximation.

This is what we mean by a **local** approximation. Our velocity estimate is fantastic for predicting the ball's height very close to `t=2`, but its quality gets worse and worse the further we extrapolate. This gives you a powerful intuition for both the utility and the limitations of the derivative.

### 🎯 Mini-Challenge: Finding the Peak

From our plots, it looks like the ball reaches its peak height at `t=5` seconds. At that single instant, the ball stops moving up and is about to start moving down. What is its velocity right at that moment?

**Part 1: Build Intuition**
First, let's get a feel for the data. Calculate the velocity for each second of the ball's flight from t=0 to t=9. Then, create a plot of these velocities vs. time. You should see a line that starts positive and ends negative, suggesting it must cross zero somewhere in the middle!

**Part 2: Find the Instantaneous Velocity**
Now, use the `plot_secant_line` function with `t=5` and a very small `h` (like `0.001`) to find the precise instantaneous velocity at the peak.

<details>
<summary>Hint: How to calculate velocity for the plot in Part 1?</summary>

You have two tools that seem relevant: `calculate_differences()` and `velocity_estimate()`. Which one is more appropriate here?

`calculate_differences(sequence)` calculates `sequence[i] - sequence[i-1]`. If your time steps are 1 second apart, this is the same as `(f(t) - f(t-1))/1`, which is a velocity estimate, but with `h=1` (or `h=-1` depending on how you view it). Is `h=1` a good, small interval for an *instantaneous* velocity estimate?

For a more accurate plot, you should probably use `velocity_estimate(height_model, t, h)` for each time `t` in your `time_intervals`, using a small `h` like `0.001`. This will give you a much better picture of the instantaneous velocity at each second.
</details>

In [ ]:
# Part 1: Plot the average velocities
# Hint: you can use the calculate_differences function from notebook 12.b!
time_intervals = list(range(11))
height_data = get_function_values(height_model, time_intervals) # We need a helper from 12.a!

# You may need to copy the calculate_differences and get_function_values functions into this cell to use them
# YOUR CODE HERE

# Part 2: Find the instantaneous velocity at t=5
# YOUR CODE HERE

<details>
<summary>Click to see a possible solution</summary>

```python
# Part 1: Plot the average velocities

# ---- Method 1: The Naive Approach (using first differences) ----
print("Method 1: Using First Differences (h=1)")
time_intervals = list(range(11))
height_data = get_function_values(height_model, time_intervals)
# This is equivalent to using h=1, which is not a great approximation
avg_velocities = calculate_differences(height_data)

plt.figure(figsize=(12, 7))
plt.plot(time_intervals, avg_velocities, 'o-', label='Velocity (using h=1)')
plt.title('Approximate Velocity vs. Time (h=1)')
plt.xlabel('Time (seconds)')
plt.ylabel('Approximate Velocity (m/s)')
plt.grid(True)
plt.axhline(0, color='black', linewidth=0.5)
plt.legend()
plt.show()

print("\nNotice that with h=1, the velocity crosses the x-axis between t=5 and t=6, not exactly at t=5. This is because our estimate is too coarse.\n")


# ---- Method 2: A Better Approach (using a small h) ----
print("Method 2: Using velocity_estimate with a small h")
time_intervals = list(range(11))
h_small = 0.0001
# Use a list comprehension to get a better velocity estimate at each point
good_velocities = [velocity_estimate(height_model, t, h_small) for t in time_intervals]

plt.figure(figsize=(12, 7))
# For comparison, let's plot both!
plt.plot(time_intervals, avg_velocities, 'o--', label='Velocity (using h=1)', alpha=0.5)
plt.plot(time_intervals, good_velocities, 'o-', label=f'Velocity (using h={h_small})')

plt.title('Comparing Velocity Estimates vs. Time')
plt.xlabel('Time (seconds)')
plt.ylabel('Estimated Velocity (m/s)')
plt.grid(True)
plt.axhline(0, color='black', linewidth=0.5)
plt.legend()
plt.show()

print("\nThis is much better! The line now clearly crosses the x-axis at t=5, which is the true peak of the trajectory.\n")


# Part 2: Find the instantaneous velocity at t=5
print('Finding the instantaneous velocity at the peak (t=5):')
plot_secant_line(t=5, h=0.001)
```

</details>

### 🤔 Discussion & Further Exploration

Here are a few questions to think about. Try to answer them by reasoning, and then feel free to write some code in new cells to test your hypotheses!

1. **Minimums vs. Maximums:** In the last challenge, you found that the instantaneous velocity was zero at the function's maximum point. How do you think finding the location of a function's *minimum* value would be similar or different?
2. **Twisty vs. Flat:** In our `height_model`, the rate of change (velocity) is itself changing at a constant rate (acceleration). What would the secant line plots look like for a function with a constant rate of change, like our `calculate_bill` function from notebook 12.b? Would the secant line's slope ever change?
3. **Higher-Order Polynomials:** What if you were analyzing a cubic function like `g(t) = t**3 - 2*t**2 + t - 1`? Do you think this method of finding the instantaneous rate of change would still work?

## 🎉 You've Reached the End!

Congratulations on completing the 12.x series on Functions, Sequences, and Plots!

You have journeyed from plotting simple functions to analyzing complex sequences and, finally, to peeking at the core concepts of calculus. You have powerful new tools—both mathematical and computational—to find patterns in the world around you.

Throughout this series, we have taught you a **numerical approach** to understanding limits and derivatives. This should give you a strong intuition for what these concepts are and how they can be practically useful. Often in the real world, we don't have a perfect equation—we only have data points from an experiment or a simulation. In those cases, the numerical methods you've just practiced are exactly how we find answers.

If you continue on your mathematical journey into a formal calculus class, you will learn the **analytical solutions** and formalisms that allow you to solve these problems with algebra instead of with a computer. We hope this series has given you a solid foundation and the curiosity to keep exploring!

Keep experimenting, stay curious, and happy coding!

---

[Return to Table of Contents](https://colab.research.google.com/github/sguy/programming-and-problem-solving/blob/main/notebooks/table-of-contents.ipynb)